In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

file_path = '/content/drive/MyDrive/datas needed/dialogues_all.csv'

In [ ]:
import pandas as pd

df = pd.read_csv(file_path)

In [ ]:
df.head()

,dialogue_id,query,response
0,0,"Hi, Mr. Smith. I'm Doctor Hawkins. Why are you...",I found it would be a good idea to get a check...
1,0,"Yes, well, you haven't had one for 5 years. Yo...",I know. I figure as long as there is nothing w...
2,0,"Well, the best way to avoid serious illnesses ...",Ok.
3,0,Let me see here. Your eyes and ears look fine....,Yes.
4,0,Smoking is the leading cause of lung cancer an...,"I've tried hundreds of times, but I just can't..."


In [ ]:
df=df[['query','response']]

In [ ]:
df = df.rename(columns={"query": "input", "response": "output"})

In [ ]:
df.shape

(61461, 2)

In [ ]:
df=df.dropna()

In [ ]:
df.shape

(56402, 2)

In [ ]:
df.head()

,input,output
0,"Hi, Mr. Smith. I'm Doctor Hawkins. Why are you...",I found it would be a good idea to get a check...
1,"Yes, well, you haven't had one for 5 years. Yo...",I know. I figure as long as there is nothing w...
2,"Well, the best way to avoid serious illnesses ...",Ok.
3,Let me see here. Your eyes and ears look fine....,Yes.
4,Smoking is the leading cause of lung cancer an...,"I've tried hundreds of times, but I just can't..."


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

In [ ]:
dataset = Dataset.from_pandas(df)


In [ ]:
# Split
dataset_split = dataset.train_test_split(test_size=0.1)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']


In [ ]:
# Load tokenizer and model
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def tokenize(example):
    text = example["input"] + tokenizer.eos_token + example["output"] + tokenizer.eos_token
    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # Add labels for training
    return tokenized


In [ ]:
tokenized_train = train_dataset.map(tokenize, remove_columns=["input", "output"])
tokenized_eval = eval_dataset.map(tokenize, remove_columns=["input", "output"])


Map:   0%|          | 0/50761 [00:00<?, ? examples/s]

Map:   0%|          | 0/5641 [00:00<?, ? examples/s]

In [ ]:
# 7. Define training arguments
training_args = TrainingArguments(
    output_dir="./dialogpt_model",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=2,
    logging_dir="./logs",
    report_to='none',
)

In [ ]:
# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,0.705100,0.712729
2,0.647300,0.688321
3,0.642200,0.676925
4,0.575100,0.673052


Epoch,Training Loss,Validation Loss
1,0.705100,0.712729
2,0.647300,0.688321
3,0.642200,0.676925
4,0.575100,0.673052
5,0.501700,0.676111


TrainOutput(global_step=63455, training_loss=0.6469556706123211, metrics={'train_runtime': 11586.0868, 'train_samples_per_second': 21.906, 'train_steps_per_second': 5.477, 'total_flos': 1.657930604544e+16, 'train_loss': 0.6469556706123211, 'epoch': 5.0})

In [ ]:
# Save final model and tokenizer
model.save_pretrained("./dialogpt_model")
tokenizer.save_pretrained("./dialogpt_model")

('./dialogpt_model/tokenizer_config.json',
 './dialogpt_model/special_tokens_map.json',
 './dialogpt_model/chat_template.jinja',
 './dialogpt_model/vocab.json',
 './dialogpt_model/merges.txt',
 './dialogpt_model/added_tokens.json',
 './dialogpt_model/tokenizer.json')